# Multi-scale simulations

**Author: Lukas Breitwieser**<br>
In this tutorial we will show how BioDynaMo support multi-scale simulations. 
Multi-scale simulation means that simulated processes happen in different time-scales---e.g. substance diffusion and neurite growth.

Let's start by setting up BioDynaMo notebooks.

In [ ]:
%jsroot on
gROOT->LoadMacro("${BDMSYS}/etc/rootlogon.C");

We define a new [standalone operation](https://biodynamo.org/docs/userguide/operation/) which only task is to print the current simulation time step if it is executed.

In [ ]:
struct TestOp : public StandaloneOperationImpl {
  BDM_OP_HEADER(TestOp);
  void operator()() override {
    auto* scheduler = Simulation::GetActive()->GetScheduler();
    auto* param = Simulation::GetActive()->GetParam();
    std::cout << "Processing iteration " 
              << scheduler->GetSimulatedSteps() 
              << " simulation time "
              << scheduler->GetSimulatedSteps() * param->simulation_time_step
              << std::endl; 
  }
};
OperationRegistry::GetInstance()->AddOperationImpl(
    "test_op", OpComputeTarget::kCpu, new TestOp());

In [ ]:
auto set_param = [](Param * param) {
    param->simulation_time_step = 2;
};
Simulation simulation("my-simulation", set_param);

Our initial model consists of one agent at origin.

In [ ]:
auto* ctxt = simulation.GetExecutionContext();
ctxt->AddAgent(new SphericalAgent());

Let's create a new instance of our class `TestOp` and add it to the scheduler.

In [ ]:
auto* op1 = NewOperation("test_op");
auto* scheduler = simulation.GetScheduler();
scheduler->ScheduleOp(op1);

Let's simulate 9 steps. We expect that `op1` will be called each time step.

In [ ]:
scheduler->Simulate(9);

Operations have a frequency attribute which specifies how often it will be executed.
An operation with frequency one will be executed at every time step; an operation with frequency two every second, and so on.

In [ ]:
op1->frequency_= 3;
scheduler->Simulate(9);

This functionality can be used to set the frequency of different processes in an agent-based model.